PROJET TUTEURE : 

Réseaux de neurones : contact Gilles Michel : gilles.michel@sudintralog.com 
Récupération sur Yahoo Finances, par API, des cours de clôture d'une liste d'actions contenues  dans un fichier CSV, depuis le 1er janvier au 31 décembre 2021. 
Stocker ces cours dans une  base de données. Utiliser un réseau de neurones sur la période du 01/01/2022 au 31/08/2022 pour prédire le cours ou le sens d'évolution de chaque action du 01/09/2022 au 31/12/2022.  
Générer un fichier CSV par action représentant une simulation de trading sur la période du  01/09/2022 au 31/12/2022 avec une base de 1 million d'euros par action au 01/09/2022. On  prendra des frais de broker de 0.01€.

In [2]:
import pandas as pd

data = {
    "Symbol": ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "BNP.PA", "AIR.PA", "OR.PA", "DTE.DE", "BMW.DE"],
    "Name": ["Apple Inc.", "Microsoft Corporation", "Alphabet Inc.", "Amazon.com Inc.", "Tesla Inc.", "BNP Paribas", "Airbus SE", "L'Oréal S.A.", "Deutsche Telekom AG", "Bayerische Motoren Werke AG (BMW)"],
    "Index": ["S&P500", "S&P500", "S&P500", "S&P500", "S&P500", "CAC40", "CAC40", "CAC40", "DAX", "DAX"]
}

actions_df = pd.DataFrame(data)

# Sauvegarder les données dans un fichier CSV
actions_df.to_csv('actions.csv', index=False)

ModuleNotFoundError: No module named 'pandas'

Étape 1 : Récupération des données
Nous devons lire le fichier CSV contenant la liste des actions et utiliser l'API Yahoo Finance pour récupérer les cours de clôture. Voici comment nous pouvons procéder :

Lire le fichier CSV :



Récupérer les données de Yahoo Finance :

In [ ]:
import yfinance as yf

# Fonction pour récupérer les données de clôture des actions
def get_stock_data(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    return hist['Close']

start_date = '2021-01-01'
end_date = '2021-12-31'
stock_data = {}

for symbol in actions_list:
    stock_data[symbol] = get_stock_data(symbol, start_date, end_date)

# Convertir les données en DataFrame
stock_data_df = pd.DataFrame(stock_data)


Stocker les données dans une base de données :

In [ ]:
from sqlalchemy import create_engine

# Créer une connexion à la base de données
engine = create_engine('sqlite:///stock_data.db')

# Stocker les données dans une table
stock_data_df.to_sql('stock_prices', con=engine, if_exists='replace', index=True)


Étape 2 : Préparation des données
Nous devons préparer les données pour l'entraînement du modèle de réseau de neurones.



Nettoyage et transformation des données :

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normaliser les données
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(stock_data_df)

# Créer des ensembles d'entraînement et de test
train_data = scaled_data[:int(len(scaled_data)*0.8)]
test_data = scaled_data[int(len(scaled_data)*0.8):]


Étape 3 : Modélisation avec un réseau de neurones
Nous allons utiliser Keras pour créer et entraîner un modèle de réseau de neurones.

Création et entraînement du modèle :

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM

def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 1
X_train, Y_train = create_dataset(train_data, look_back)
X_test, Y_test = create_dataset(test_data, look_back)

# Reshape des données pour LSTM [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=100, batch_size=1, verbose=2)

# Prédictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)


Étape 4 : Simulation de trading
Nous allons utiliser les prédictions pour simuler des transactions de trading.

Simulation de trading et génération de fichiers CSV

In [ ]:
initial_capital = 1000000
broker_fee = 0.01

def simulate_trading(predictions, initial_capital):
    capital = initial_capital
    for i in range(len(predictions)-1):
        # Acheter à la prédiction actuelle et vendre à la prochaine
        if predictions[i+1] > predictions[i]:
            # Acheter
            shares = capital / predictions[i]
            capital = shares * predictions[i+1] - broker_fee
        else:
            capital -= broker_fee
    return capital

trading_results = {}

for symbol in stock_data_df.columns:
    predictions = model.predict(stock_data_df[symbol].values.reshape(-1, 1))
    final_capital = simulate_trading(predictions, initial_capital)
    trading_results[symbol] = final_capital

# Sauvegarder les résultats dans des fichiers CSV
for symbol, capital in trading_results.items():
    result_df = pd.DataFrame({'Symbol': [symbol], 'Final Capital': [capital]})
    result_df.to_csv(f'results_{symbol}.csv', index=False)
